## HERMES - furosemide phenotype

#### Clone HeRmes repository

In [ ]:
#system("git clone https://github.com/nicksunderland/heRmes.git")

#### Pushing changes to Github

In [ ]:
# in ther terminal run
# cd /opt/notebooks
# dx download git_push.sh
# run bash git_push.sh 

#### Project & record ID

In [ ]:
projectid <- "project-GvZyZ20J81vgPJGbJy8pgpyq"
recordid  <- "record-Gvb0Bg0Jfxfv0q8Fb2pXqKjg"

#### Libraries

In [ ]:
library(glue)
library(data.table)
library(yaml)
source("/opt/notebooks/heRmes/R/ukbb_extraction_utils.R")

## Extract data
Given the large datasets we make use of the `dx run table-exporter` to extract the required phenotype data. The extraction function below will create a table-exportor job which you will be able to track in the 'Monitor' table on your RAP's homepage. The data will be extracted to your project into a folder called `hermes3_data`. The data is not immediately uploaded to this session, although we will import it later. 

To get the small data dictionaries locally in this session we use the `dx extract_dataset` function. 

#### Download data dictionary

In [ ]:
setwd("/opt/notebooks")
dataset <- glue("{projectid}:{recordid}")
cmd <- glue("dx extract_dataset {dataset} -ddd")
system(cmd)
dict_files <- list.files(pattern="codings|data_dictionary|entity_dictionary")
data_dict_file <- dict_files[grepl("data_dictionary", dict_files)]

#### Read the extraction config file

In [ ]:
config <- read_yaml("/opt/notebooks/heRmes/scripts/ukbb_extraction_config.yml")

extraction_template <- lapply(config, function(table) filter_ukbb_data_dict(data_dict_file, table$entity, table$columns))
                       
lapply(extraction_template, head, 3)               

#### Run Table-Exporter extraction
Check the monitor tab in the project for completion of the file extraction jobs

In [ ]:
for (table in names(config)) {
    
    filepath <- file.path("/mnt/project", config[[table]][["output"]]) 
    if (!file.exists(filepath)) {
        extract_ukbb_data(dataset = dataset, 
                          fields  = extraction_template[[table]][["name"]],
                          entity  = config[[table]][["entity"]],
                          output  = config[[table]][["output"]])
    } else {
        cat("File:", filepath, "- exists\n")
    }
    
}